In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [ ]:
#Import Data
df = pd.read_csv('C:/Kuliah/Bangkit/Capstone/Dataset/Labeling/[PREPROPPED]_Playstore.csv',sep=',')
#df = df[["title",'sentiment']]

In [ ]:
reviews = df['title'].values
labels = df['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [ ]:
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 5
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/5
437/437 [==============================] - 34s 78ms/step - loss: 0.1874 - accuracy: 0.9291 - val_loss: 0.2217 - val_accuracy: 0.9215
Epoch 2/5
437/437 [==============================] - 35s 80ms/step - loss: 0.1414 - accuracy: 0.9470 - val_loss: 0.2170 - val_accuracy: 0.9202
Epoch 3/5
437/437 [==============================] - 33s 75ms/step - loss: 0.1112 - accuracy: 0.9611 - val_loss: 0.2539 - val_accuracy: 0.9183
Epoch 4/5
437/437 [==============================] - 35s 80ms/step - loss: 0.0873 - accuracy: 0.9710 - val_loss: 0.2659 - val_accuracy: 0.9138
Epoch 5/5
437/437 [==============================] - 36s 83ms/step - loss: 0.0689 - accuracy: 0.9779 - val_loss: 0.2924 - val_accuracy: 0.9176


In [ ]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

162/162 [==============================] - 4s 22ms/step
Accuracy of prediction on test set :  0.9200463499420626


In [ ]:
# Assuming you have already initialized 'tokenizer' and 'max_length'

# Load the data
df = pd.read_csv('[LABELLED]_LinkedIn.csv', sep=',')
df = df[["text_data"]]

# Convert to a sequence
sequences = tokenizer.texts_to_sequences(df["text_data"])

# Pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get labels based on probability 1 if p >= 0.5 else 0
prediction = model.predict(padded)

# Convert probabilities to labels
pred_labels = [1 if p >= 0.5 else 0 for p in prediction]

# Create a new DataFrame with the original text data and predicted sentiments
result_df = pd.DataFrame({"text_data": df["text_data"], "predicted_sentiment": pred_labels})

# Display predictions
for i in range(len(result_df)):
    print(result_df["text_data"].iloc[i])
    print("Predicted sentiment: ", "Positive" if result_df["predicted_sentiment"].iloc[i] == 1 else "Negative")
    print()

# Save the results to a new CSV file
result_df.to_csv('predicted_sentiments.csv', index=False)


30/30 [==============================] - 1s 26ms/step
captcha tidak valid padahal sudah memilih jawaban yang benar
Predicted sentiment:  Negative

Aplikasi Lucu, dia bilang gratis uji coba satu bulan. eh giliran mau nyoba dia minta transfer. jangan bohoong lah. kalau gak gratis uji coba ya bilang aja dari awal
Predicted sentiment:  Negative

Setiap buka aplikasi LinkedIn langsung keluar lagi buka lagi aplikasi nya keluar lagi kesel banget
Predicted sentiment:  Negative

Sudah pernah masuk tetapi sudah lama. Tentu saja menggunakan nama sendiri lebih baik.
Predicted sentiment:  Negative

Apps yg berguna untuk cari temen, dan nqmbah channel... peluang kerjanya jg bagus krn ga semua otng tau
Predicted sentiment:  Positive

Cara loginnya gmna ini bikin emosi loh
Predicted sentiment:  Negative

Social media yg bagus untuk mencari kerja
Predicted sentiment:  Positive

Tolong diperbaiki lagi karena masih sangat banyak masalah. Mulai dari Upload foto banner tapi malah lari ke foto profile, masa